In [288]:
import pandas as pd
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from nba_api.stats.static import teams as static_teams
from nba_api.stats.endpoints import BoxScoreFourFactorsV3, TeamGameLog, LeagueStandingsV3
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
from requests.exceptions import ReadTimeout, ConnectionError
import random, math
import pathlib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score


RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Linear Regression
### Goal: Predict Final Record of NBA Teams
First, I will be using the Four Factors statistics (Effective Field Goal Percentage, Free Throw Rate, Turnover Percentage, Offensive Rebound Rate,
Opponent Effective Field Goal Percentage, Opponent Free Throw Rate, Opponent Turnover Percentage, Opponent rebound Rate), which are established to
be excellent predictors of team success, with a Linear Regression model trained on the first n games of a season, and then test on a different season
given the Four Factor stats from the first n games of that season. 

In [289]:
COMMON_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Origin": "https://www.nba.com",
    "Referer": "https://www.nba.com/",
}

CACHE_DIR = pathlib.Path("cache_pkl")
CACHE_DIR.mkdir(exist_ok=True)
NETWORK_CALLS = 0

# Reusable method to pull data from an endpoint of NBA_API, checks if the data has already been saved locally
def endpoint(endpoint_cls, *, cache_key=None, max_retries=3, base_sleep=1.5, **kwargs):
    global NETWORK_CALLS

    # Check if the data has already been retrieved
    if cache_key:
        path = CACHE_DIR / f"{cache_key}.pkl"
        if path.exists():
            frames = pd.read_pickle(path)

            stub = type("Stub", (), {})()
            stub.get_data_frames = lambda f=frames: f
            stub.team_stats = type("DS", (), {
                "get_data_frame": staticmethod(lambda df=frames[1] if len(frames) > 1 else frames[0]: df)
            })()
            stub._from_cache = True
            return stub
    
    # If not pull it from the API
    for attempt in range(max_retries):
        try:
            obj = endpoint_cls(
                headers=COMMON_HEADERS,
                timeout=60,
                **kwargs
            )
            NETWORK_CALLS += 1
            # Save the data to speed up subsequent program runs
            if cache_key:
                pd.to_pickle(obj.get_data_frames(),
                             CACHE_DIR / f"{cache_key}.pkl")
                    
            obj._from_cache = False
            return obj
        
        except (ReadTimeout, ConnectionError):
            wait = base_sleep * (2 ** attempt) + random.uniform(0, 0.5)
            print(f"{endpoint_cls.__name__} retry {attempt+1}/{max_retries}"
                  f" - sleeping {wait:.1f}s")
            time.sleep(wait)
            
    # Error if we hit max retries
    raise RuntimeError(f"{endpoint_cls.__name__} failed after {max_retries} retries")
            

In [290]:
# Function to get the game IDs from the training set
def get_game_ids(team_id, season, n_games):
    games = endpoint(
        TeamGameLog,
        cache_key=f"gamelog_{team_id}_{season}",
        team_id=team_id,
        season=season,
        season_type_all_star="Regular Season"
        ).get_data_frames()[0]
    return games.sort_values("GAME_DATE")["Game_ID"].head(n_games).tolist()

In [291]:


# Features we will be using
FEATURES = [
    "effectiveFieldGoalPercentage",
    "freeThrowAttemptRate",
    "teamTurnoverPercentage",
    "offensiveReboundPercentage",
    "oppEffectiveFieldGoalPercentage",
    "oppFreeThrowAttemptRate",
    "oppTeamTurnoverPercentage",
    "oppOffensiveReboundPercentage"
]

# Speed up data retrieval
MAX_THREADS = 4
PER_REQUEST_SLEEP = 0.5


# Get the features from the API endpoint
def get_features(game_id, team_id):
        stats = endpoint(
            BoxScoreFourFactorsV3,
            cache_key=f"bsff_{game_id}",
            game_id=game_id,
            start_period=1, start_range=0,
            end_period=1, end_range=0,
            range_type=0)
        team_df = stats.team_stats.get_data_frame()
        row = team_df.loc[team_df["teamId"] == team_id, FEATURES]
        # Avoid making the API mad
        if not getattr(stats, "_from_cache", False):
            time.sleep(PER_REQUEST_SLEEP)
        return row.squeeze().astype(float)
    


In [292]:
def combine_rows(team_id, season, n_games):
    ids = get_game_ids(team_id, season, n_games)
    rows = []
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as pool:
        futures = [pool.submit(get_features, id, team_id) for id in ids]
        for future in as_completed(futures):
            rows.append(future.result())
    
    # Return the mean features for n_games
    return pd.concat(rows, axis=1).mean(axis=1)

In [293]:
def generate_df(season, n_games):
    records = []
    for team in static_teams.get_teams():
        row = combine_rows(team["id"], season, n_games)
        row["TEAM_NAME"] = team["full_name"]
        row["TEAM_ID"] = team["id"]
        records.append(row)
    return pd.DataFrame(records)

In [294]:
# Get the number of wins for each team
def get_wins(season, season_type="Regular Season"):
    df = endpoint(
        LeagueStandingsV3,
        cache_key=f"stand_{season}",
        season=season,
        season_type=season_type
    ).get_data_frames()[0]
    
    df = df.rename(columns={"TeamID" : "TEAM_ID",
                            "TeamName" : "TEAM_NAME",
                            "WINS" : "W"})
    return df.drop(columns=["TEAM_NAME"])

In [295]:
def generate_season_matrix(season, n_games):
    features = generate_df(season, n_games)
    wins = get_wins(season)
    df = features.merge(wins, on="TEAM_ID", how="inner")
    df.dropna(subset=FEATURES, inplace=True)
    return df
    

In [296]:
# Function to fit a linear regression model to the training data
def train_lr_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [297]:
# Function to valuate the fit of the Linear Regression model
def evaluate_model_lr(model, X_test, y_test):
    score = model.score(X_test, y_test)
    print(f"R^2 Stat on test set: {score: .4f}")

In [298]:
# Plot with Plotly for interactivity
def plotly_plot_lr(df, model_name, actual_col="Actual_Wins", predicted_col="Predicted_Wins"):
    fig = px.scatter(
        df,
        x=actual_col,
        y=predicted_col,
        hover_data=["TEAM_NAME"],
        labels={actual_col: "Actual Wins", predicted_col: "Predicted Wins"},
        title=f"({model_name}) Actual Wins vs. Predicted Wins (Test Season: 2023-24)"
    )
    
    # Plot regression
    min_val = min(df[actual_col].min(), df[predicted_col].min())
    max_val = max(df[actual_col].max(), df[predicted_col].max())
    fig.add_shape(
        type="line",
        x0=min_val, y0=min_val, x1=max_val, y1=max_val,
        line=dict(color="red", dash="dash")
    )
    
    fig.show()

In [299]:
# Generate final team record predictions from Linear Regression Model
def predict_final_record_lr(model, df, features=FEATURES):

    X_current = df[features]
    predicted_wins = model.predict(X_current)

    results_df = pd.DataFrame({
        "TEAM_NAME": df["TEAM_NAME"],
        "Predicted_Wins": predicted_wins
    })

    # Round prediction to nearest integer
    results_df["Predicted_Wins"] = results_df["Predicted_Wins"].round().astype(int)
    results_df["Predicted_Losses"] = 82 - results_df["Predicted_Wins"]
    
    return results_df

In [300]:
N_GAMES = 20
# Train Model
train_df = generate_season_matrix("2022-23", N_GAMES)
X_train = train_df[FEATURES]
y_train = train_df["W"]

model = train_lr_model(X_train, y_train)

test_df = generate_season_matrix("2023-24", N_GAMES)
X_test = test_df[FEATURES]
y_test = test_df["W"]


# Generate Predictions
y_pred = model.predict(X_test).round()

result = test_df[["TEAM_NAME", "W"]].copy()
result["Predicted_Wins"] = y_pred.astype(int)
result["Predicted_Losses"] = 82 - result["Predicted_Wins"]

print("Network Calls: ", NETWORK_CALLS)
# Get R^2
r2 = model.score(X_test, y_test)

print(f"R2 on 2023-2024 data: {r2:.2f}")

# Generate plot
#plotly_plot_lr(result, actual_col="W", predicted_col="Predicted_Wins")

# Print chart of Predicted vs. Actual Wins
show_df = result.rename(columns={"W" : "Actual_Wins"}).sort_values("Actual_Wins", ascending=False)
print(show_df.to_string(index=False))


Network Calls:  0
R2 on 2023-2024 data: 0.76
             TEAM_NAME  Actual_Wins  Predicted_Wins  Predicted_Losses
        Boston Celtics           64              69                13
        Denver Nuggets           57              60                22
 Oklahoma City Thunder           57              59                23
Minnesota Timberwolves           56              53                29
  Los Angeles Clippers           51              50                32
      Dallas Mavericks           50              46                36
       New York Knicks           50              43                39
          Phoenix Suns           49              44                38
       Milwaukee Bucks           49              54                28
  New Orleans Pelicans           49              48                34
   Cleveland Cavaliers           48              43                39
        Indiana Pacers           47              47                35
    Los Angeles Lakers           47          

In [301]:
'''def fit_model_and_evaluate(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    r2 = model.score(X_test, y_test)
    print(f"{name}:  R^2 Score: {r2:.3f}")
    result_df = predict_final_record_lr(model, test_df)
    result_df["Actual_Wins"] = y_test.values

    plotly_plot_lr(result_df)
    
    return model'''

'def fit_model_and_evaluate(name, model, X_train, y_train, X_test, y_test):\n    model.fit(X_train, y_train)\n    r2 = model.score(X_test, y_test)\n    print(f"{name}:  R^2 Score: {r2:.3f}")\n    result_df = predict_final_record_lr(model, test_df)\n    result_df["Actual_Wins"] = y_test.values\n\n    plotly_plot_lr(result_df)\n\n    return model'

In [302]:
#ols = fit_model_and_evaluate("Linear Regression", LinearRegression(), X_train, y_train, X_test, y_test)


### Linear Model Evaluation
It appears that, at least with a linear model, Effective Field Goal Percentage is by far the most important feature being used in the predictions, followed by Opponent Effective Field Goal Percentage. This makes sense as points scored vs points allowed would be the most directly linked variable related to the outcome of games, and efficiency in scoring and defending should make a big difference in that.

The coefficients show that the features are strongly correlated with wins (with the exception of free throw rates).

In [303]:
import statsmodels.api as sm

X_sm = sm.add_constant(train_df[FEATURES])
ols = sm.OLS(train_df["W"], X_sm).fit()
print(ols.summary())



                            OLS Regression Results                            
Dep. Variable:                      W   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.670
Method:                 Least Squares   F-statistic:                     8.362
Date:                Mon, 05 May 2025   Prob (F-statistic):           4.59e-05
Time:                        16:12:39   Log-Likelihood:                -89.735
No. Observations:                  30   AIC:                             197.5
Df Residuals:                      21   BIC:                             210.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [304]:

# Generic function to fit a model, plot it, and generate a bunch of info
def evaluate_model(name, model, X_train, y_train, X_test, y_test, test_df, plot=True):
    model.fit(X_train, y_train)
    
    y_hat = model.predict(X_test)
    
    # Check performance on the Training data to check for overfitting
    y_hat_train = model.predict(X_train)
    train_r2 = r2_score(y_train, y_hat_train)
    
    # Model Metrics:
    r2 = r2_score(y_test, y_hat)
    mae = mean_absolute_error(y_test, y_hat)
    rmse = root_mean_squared_error(y_test, y_hat)
    medae = median_absolute_error(y_test, y_hat)
    evs = explained_variance_score(y_test, y_hat)
    
    '''if plot:
        plot_fit = predict_final_record_lr(model, test_df)\
            .merge(test_df[["TEAM_NAME", "W"]], on="TEAM_NAME")\
            .rename(columns={"W": "Actual_Wins"})
        plotly_plot_lr(plot_fit, actual_col="Actual_Wins", predicted_col="Predicted_Wins")'''
        
    return {
        "Model" : name,
        "Training R2" : train_r2,
        "R2" : r2,
        "MAE" : mae,
        "RMSE" : rmse,
        "MedAE" : medae,
        "ExpVar" : evs
    }
    

In [305]:
train_seasons = ["2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]
test_season = "2023-24"

def generate_multi_season_matrix(seasons=train_seasons, n_games=N_GAMES):
    dfs = []
    for season in seasons:
        df_season = generate_season_matrix(season, n_games)
        df_season["season"] = season
        dfs.append(df_season)
    return pd.concat(dfs, ignore_index=True)

multi_season_df = generate_multi_season_matrix()
X_multi = multi_season_df[FEATURES]
y_multi = multi_season_df["W"]




In [306]:
# Improved Train/Test/Validation Splitting function
def split_data(seasons=train_seasons, test_season="2023-24", val_size=0.2):
    # Generate datasets
    train_val_df = generate_multi_season_matrix(seasons=seasons, n_games=N_GAMES)
    test_df = generate_season_matrix(test_season, N_GAMES)
    
    # Split data randomly
    split_idx = np.random.RandomState(RANDOM_SEED).rand(len(train_val_df)) > val_size
    train_df = train_val_df[split_idx]
    val_df = train_val_df[~split_idx]
    
    # Get features and target
    X_train = train_df[FEATURES]
    y_train = train_df['W']
    
    X_val = val_df[FEATURES]
    y_val = val_df['W']
    
    X_test = test_df[FEATURES]
    
    return (X_train, y_train), (X_val, y_val), (X_test, test_df['W']), test_df
    
    
    

In [307]:
(X_train, y_train), (X_val, y_val), (X_test, test_df['W']), test_df = split_data()
# Improved model training and evaluation function
def fit_model_and_evaluate(name, model, X_train, y_train, X_val, y_val, X_test, y_test):
    # Fit model
    model.fit(X_train, y_train)
    
    # Get validation and test scores
    val_score = model.score(X_val, y_val)
    test_score = model.score(X_test, y_test)
    
    print(f"{name} Scores: ")
    print(f"Validation R^2: {val_score:.3f}")
    print(f"Test R^2: {test_score:.3f}")
    
    y_pred = model.predict(X_test)
    result_df = pd.DataFrame({
        "TEAM_NAME" : test_df["TEAM_NAME"],
        "Predicted_Wins" :y_pred.round().astype(int),
        "Actual_Wins" : y_test
    })
    result_df["Predicted_Losses"] = 82 - result_df["Predicted_Wins"]
    plotly_plot_lr(result_df, model_name=name)
    
    
    return model
    
    


In [308]:
# Function to rank the importance of each Feature in Linear Models
def get_importance(model, features):
    if hasattr(model, 'named_steps'):
        for step_name, step in model.named_steps.items():
            if hasattr(step, 'coef_') or hasattr(step, 'feature_importances_'):
                model = step
                break
    if hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
    elif hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        return -1
        
    
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    return importance_df



# Linear Regression
Now I will fit an Ordinary Least Squares Linear Regression Model and see what our baseline performance is with the features.

### Measurements:
R^2:                       0.762
Mean Absolute Error:       4.709
Root Mean Squared Error:   6.426
Median Absolute Error:     3.120
Explained Variance:        0.771

### Analysis:
R^2: of 0.762 (76.2% of the variance in NBA team wins explained by our model) is a very solid start. This suggests that the features I am working with are very good predictors of team success, even when only getting a quarter of the data.

MAE: Mean absolute error of 4.71 wins per team.

RMSE: 6.43, slightly higher due to a couple of outliers that were way off. i.e. 76ers predicted wins.

MedAE: 3.29, 50% of predictions are within 3.12 wins.

ExpVar: 0.77, checking if it was close to R^2 as it should be.


# New Linear Regression

In [309]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler

# Generate our Linear Regression Model
'''lr_pipe = make_pipeline(StandardScaler(), LinearRegression())
lr_model = evaluate_model("Linear Regression", lr_pipe, X_train, y_train, X_test, y_test, test_df)
display(lr_model)'''

lr = make_pipeline(StandardScaler(), LinearRegression())
lr_model = fit_model_and_evaluate("Linear Regression", lr, X_train, y_train, X_val, y_val, X_test, y_test=test_df['W'])
lr_importance = get_importance(lr_model, FEATURES)
print(lr_importance)


Linear Regression Scores: 
Validation R^2: 0.594
Test R^2: 0.743


                           feature  importance
0     effectiveFieldGoalPercentage    5.646507
4  oppEffectiveFieldGoalPercentage    4.898163
2           teamTurnoverPercentage    2.866725
3       offensiveReboundPercentage    2.397702
6        oppTeamTurnoverPercentage    1.846628
7    oppOffensiveReboundPercentage    1.804693
1             freeThrowAttemptRate    0.667044
5          oppFreeThrowAttemptRate    0.126075


# Ridge Regression
Now I will fit a Ridge Regression model to compare to the OLS model. This will standardize the features and correct for large amounts of colinearity.

### Measurements:
R^2:                       0.767
Mean Absolute Error:       4.581
Root Mean Squared Error:   6.3668
Median Absolute Error:     3.288
Explained Variance:        0.775

### Analysis:
R^2: of 0.767 (76.7% of the variance in NBA team wins explained by our model) is slightly higher than the OLS Linear Regression Model. This suggests that the regularization and alphas made a small positive impact on the accuracy of the model.

MAE: Mean absolute error of 4.6 wins per team.

RMSE: 6.37, slightly higher due to a couple of outliers that were way off. i.e. 76ers predicted wins.

MedAE: 3.29, 50% of predictions are within 3.29 wins (a slightly wider margin than the OLS model).

ExpVar: 0.77, checking if it was close to R^2 as it should be.



In [310]:
alphas = np.logspace(-4, 2, 50)


# Generate our Ridge Regression model, find the best alpha
ridge_pipe = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas, cv=5))
ridge = evaluate_model("Ridge Regression", ridge_pipe, X_train, y_train, X_test, y_test, test_df)
display(ridge)

{'Model': 'Ridge Regression',
 'Training R2': 0.5637603520720336,
 'R2': 0.725161531071632,
 'MAE': 5.779859670966304,
 'RMSE': 6.912687499398884,
 'MedAE': np.float64(5.651379249580151),
 'ExpVar': 0.7404292937692285}

# New Ridge Regression Implementation w/ Multi Season and Scaling

In [311]:
ridge = make_pipeline(StandardScaler(), RidgeCV(alphas=np.logspace(-4, 2, 50), cv=5))
ridge_model = fit_model_and_evaluate("Ridge Regression", ridge, X_train, y_train, X_val, y_val, X_test, y_test=test_df['W'])
print("\nRidge Regression Feature Importance Breakdown:")
print(get_importance(ridge_model, FEATURES))

Ridge Regression Scores: 
Validation R^2: 0.596
Test R^2: 0.725



Ridge Regression Feature Importance Breakdown:
                           feature  importance
0     effectiveFieldGoalPercentage    5.344847
4  oppEffectiveFieldGoalPercentage    4.677284
2           teamTurnoverPercentage    2.646079
3       offensiveReboundPercentage    2.166186
7    oppOffensiveReboundPercentage    1.721174
6        oppTeamTurnoverPercentage    1.690791
1             freeThrowAttemptRate    0.589963
5          oppFreeThrowAttemptRate    0.105987


Ridge Regression Results:
Model Metrics:



# Lasso Regression:

In [312]:
lasso = make_pipeline(StandardScaler(), LassoCV(
    alphas=np.logspace(-4, 2, 50),
    cv=5,
    random_state=RANDOM_SEED
))
lasso_model = fit_model_and_evaluate("Lasso", lasso, X_train, y_train, X_val, y_val, X_test, y_test=test_df['W'])
lasso_importance = get_importance(lasso_model, FEATURES)

Lasso Scores: 
Validation R^2: 0.590
Test R^2: 0.714


# Random Forest

In [313]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=5, min_samples_leaf=4, bootstrap=True, oob_score=True, random_state=RANDOM_SEED)
rf = fit_model_and_evaluate("Random Forest", rf_model, X_train, y_train, X_val, y_val, X_test, y_test=test_df["W"])

Random Forest Scores: 
Validation R^2: 0.466
Test R^2: 0.477


In [314]:
from sklearn.ensemble import HistGradientBoostingRegressor

boosted_tree_model = HistGradientBoostingRegressor(learning_rate=0.01, max_depth=3, max_iter=1000, random_state=RANDOM_SEED, early_stopping=False, min_samples_leaf=1, validation_fraction=0.2)
boosted_tree = fit_model_and_evaluate("Boosted Tree", boosted_tree_model, X_train, y_train, X_val, y_val, X_test, y_test=test_df['W'])

Boosted Tree Scores: 
Validation R^2: 0.466
Test R^2: 0.397


In [315]:
# Get Feature Importance
feat_importance = pd.DataFrame({
    'feature' : FEATURES,
    'importance' : rf_model.feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature Importances: ")
print(feat_importance)


Feature Importances: 
                           feature  importance
0     effectiveFieldGoalPercentage    0.407624
4  oppEffectiveFieldGoalPercentage    0.335279
2           teamTurnoverPercentage    0.074003
3       offensiveReboundPercentage    0.049124
6        oppTeamTurnoverPercentage    0.038123
5          oppFreeThrowAttemptRate    0.032947
1             freeThrowAttemptRate    0.031796
7    oppOffensiveReboundPercentage    0.031104
